In [1]:
import pandas as pd
import numpy as np
from dateutil import parser
import re
from fuzzywuzzy import process, fuzz
from sklearn.model_selection import train_test_split
from datetime import datetime
import json

In [2]:
TRAIN_DATA_ADDRESS = "datathon-dataset/test_x.csv"
train_data = pd.read_csv(TRAIN_DATA_ADDRESS)
train_data

,Basvuru Yili,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Bölüm,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2023,Kadın,21.06.2002 00:00,Amasya,Amasya,Munzur Üniversitesi,Devlet,NaN,Hayır,Sosyal Hizmet,...,Takım Oyuncusu,Evet,Kızılay da gönüllü dağıtım,NaN,Hayır,-,Evet,NaN,NaN,0
1,2023,Kadın,11.05.2004 00:00,Konya,Konya,Hacettepe Üniversitesi,Devlet,NaN,Hayır,Uluslararası İlişkiler,...,Bireysel Spor,Hayır,-,NaN,Hayır,-,Evet,NaN,NaN,1
2,2023,Erkek,24.04.2002 00:00,İstanbul,İstanbul,Kapadokya Üniversitesi,Özel,NaN,Evet,Diğer,...,Bireysel Spor,Hayır,-,NaN,Hayır,-,Evet,NaN,NaN,2
3,2023,Erkek,11.03.2003 00:00,Mardin,Mardin,Mardin Artuklu Üniversitesi,Devlet,NaN,Hayır,İslami İlimler,...,-,Evet,Lise gezilerde liderlik yapıyordum,NaN,Hayır,-,Hayır,NaN,NaN,3
4,2023,Erkek,26.01.2002 00:00,Samsun,İstanbul,Boğaziçi Üniversitesi,Devlet,NaN,Hayır,İşletme,...,-,Evet,"Toplum Gönüllüleri Vakfı'nda ""Yol Arkadaşım"" P...",NaN,Evet,"12 aydır, yolculuğuna bir yıl önce başlamış ol...",Evet,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,Kadın,3.01.2002 00:00,Yozgat,Ankara,Gazi Üniversitesi,Devlet,NaN,Hayır,Endüstri Mühendisliği,...,-,Evet,TOG Vakfı'nda yaklaşık bir yıldır hem aktif gö...,NaN,Hayır,-,Evet,NaN,NaN,11044
11045,2023,Kadın,9.09.2001 00:00,Konya,Konya,Niğde Ömer Halisdemir Üniversitesi,Devlet,NaN,Hayır,Gıda Mühendisliği,...,-,Evet,Konyada düzenlenen 5. İslami dayanışma oyunlar...,NaN,Hayır,-,Hayır,NaN,NaN,11045
11046,2023,Erkek,8.06.2004 00:00,Ankara,Ankara,Karabük Üniversitesi,Devlet,NaN,Hayır,Elektrik-Elektronik Mühendisliği,...,Takım Oyuncusu,Evet,Hayvanlara yardım adı altında hayvanlara yardı...,NaN,Hayır,-,Evet,NaN,NaN,11046
11047,2023,Kadın,16.10.2001 00:00,Adıyaman,İstanbul,Bursa Uludağ Üniversitesi,Devlet,NaN,Hayır,"Moleküler Biyoloji, Genetik ve Biyomühendislik",...,Bireysel Spor,Evet,"Ağaç dikme, huzur evi ziyareti",NaN,Evet,Eğitim olarak deneyimim var. Aynı zamanda “Asp...,Evet,NaN,NaN,11047


In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11049 entries, 0 to 11048
Data columns (total 43 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               11049 non-null  int64  
 1   Cinsiyet                                                   11049 non-null  object 
 2   Dogum Tarihi                                               11049 non-null  object 
 3   Dogum Yeri                                                 11049 non-null  object 
 4   Ikametgah Sehri                                            11049 non-null  object 
 5   Universite Adi                                             11049 non-null  object 
 6   Universite Turu                                            11049 non-null  object 
 7   Burslu ise Burs Yuzdesi                                    0 non-null      float64
 8   Burs A

In [4]:
def modify_column_names(col):
    col = col.strip()
    col = col.lower()
    col = col.replace(" ", "_")
    return col

In [5]:
train_data.columns = [modify_column_names(col) for col in train_data.columns]

In [6]:
columns_to_drop = [
    "lise_adi_diger",
    "lise_turu",
    "lise_bolum_diger",
    "hangi_stk'nin_uyesisiniz?",
    "daha_once_baska_bir_universiteden_mezun_olmus",
    "daha_önceden_mezun_olunduysa,_mezun_olunan_üniversite",
    "uye_oldugunuz_kulubun_ismi",
    "spor_dalindaki_rolunuz_nedir?",
    "girisimcilikle_ilgili_deneyiminiz_var_mi?",
    "baska_bir_kurumdan_burs_aliyor_mu?",
    "burs_aldigi_baska_kurum",
    "baska_kurumdan_aldigi_burs_miktari",
    "lise_bolumu",
    "girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz?",
]

In [7]:
train_data.drop(columns=columns_to_drop, inplace=True)

In [8]:
for column in train_data.columns:
    if train_data[column].dtype.name == "object":
        train_data[column] = train_data[column].astype("str")

In [9]:
train_data.describe(include="all")

,basvuru_yili,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,...,baba_calisma_durumu,baba_sektor,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id
count,11049.0,11049,11049,11049,11049,11049,11049,0.0,11049,11049,...,11049,11049,11049.000000,11049,11049,11049,0.0,11049,0.0,11049.000000
unique,NaN,3,2135,83,83,203,2,NaN,2,349,...,2,4,NaN,2,2,2,NaN,2,NaN,NaN
top,NaN,Kadın,1.01.2002 00:00,İstanbul,İstanbul,İstanbul Teknik Üniversitesi,Devlet,NaN,Hayır,Bilgisayar Mühendisliği,...,Evet,-,NaN,Hayır,Hayır,Evet,NaN,Evet,NaN,NaN
freq,NaN,6378,135,1814,3301,322,9215,NaN,9400,817,...,6177,4752,NaN,5940,6422,6270,NaN,8863,NaN,NaN
mean,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.118563,NaN,NaN,NaN,NaN,NaN,NaN,5524.000000
std,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.256795,NaN,NaN,NaN,NaN,NaN,NaN,3189.715896
min,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,2762.000000
50%,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,NaN,NaN,5524.000000
75%,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN,NaN,8286.000000


In [10]:
train_data.columns

Index(['basvuru_yili', 'cinsiyet', 'dogum_tarihi', 'dogum_yeri',
       'ikametgah_sehri', 'universite_adi', 'universite_turu',
       'burslu_ise_burs_yuzdesi', 'burs_aliyor_mu?', 'bölüm',
       'universite_kacinci_sinif', 'universite_not_ortalamasi', 'lise_adi',
       'lise_sehir', 'lise_mezuniyet_notu', 'anne_egitim_durumu',
       'anne_calisma_durumu', 'anne_sektor', 'baba_egitim_durumu',
       'baba_calisma_durumu', 'baba_sektor', 'kardes_sayisi',
       'girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?',
       'profesyonel_bir_spor_daliyla_mesgul_musunuz?',
       'aktif_olarak_bir_stk_üyesi_misiniz?', 'stk_projesine_katildiniz_mi?',
       'ingilizce_biliyor_musunuz?', 'ingilizce_seviyeniz?', 'id'],
      dtype='object')

In [11]:
if "degerlendirme_puani" in train_data.columns:
    train_data["degerlendirme_puani"].fillna(0, inplace=True)

In [12]:
cinsiyet_map = {
    "Erkek": 0,
    "Kadın": 1,
    "Belirtmek istemiyorum": 2,
    "ERKEK": 0,
    "nan": 3,
}

In [13]:
train_data["cinsiyet"] = train_data["cinsiyet"].map(cinsiyet_map)

In [14]:
train_data["cinsiyet"].value_counts(dropna=False)

cinsiyet
1    6378
0    4654
2      17
Name: count, dtype: int64

In [15]:
train_data["dogum_tarihi"].value_counts()

dogum_tarihi
1.01.2002 00:00     135
1.01.2001 00:00      98
1.01.2003 00:00      87
1.01.2000 00:00      78
1.01.2004 00:00      54
                   ... 
10.05.1999 00:00      1
29.01.2004 00:00      1
16.07.2003 00:00      1
5.11.1999 00:00       1
1.10.2005 00:00       1
Name: count, Length: 2135, dtype: int64

In [16]:
def parse_date(date_string):
    try:
        result = parser.parse(date_string, dayfirst=True)
        if (
            result.year > datetime.now().year
            or result.year <= datetime.now().year - 100
        ):
            raise ValueError()
        else:
            return result
    except ValueError:
        return pd.NA

In [17]:
train_data["dogum_tarihi"] = train_data["dogum_tarihi"].apply(parse_date)

In [18]:
train_data["dogum_tarihi"] = pd.to_datetime(
    train_data["dogum_tarihi"], format="%Y-%m-%d %H:%M:%S"
)

In [19]:
train_data["dogum_tarihi"].isna().sum()

0

In [20]:
train_data["dogum_yili"] = train_data["dogum_tarihi"].dt.year
train_data["dogum_ayi"] = train_data["dogum_tarihi"].dt.month
train_data.drop(columns="dogum_tarihi", inplace=True)

In [21]:
train_data["dogum_yili"].fillna(train_data["dogum_yili"].mode()[0], inplace=True)
train_data["dogum_ayi"].fillna(train_data["dogum_ayi"].mode()[0], inplace=True)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16198/3503408563.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["dogum_yili"].fillna(train_data["dogum_yili"].mode()[0], inplace=True)
/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16198/3503408563.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on

In [22]:
train_data["dogum_yeri"].unique()

array(['Amasya', 'Konya', 'İstanbul', 'Mardin', 'Samsun', 'Isparta',
       'Hatay', 'Ordu', 'Adıyaman', 'Bursa', 'Tekirdağ', 'Mersin',
       'Kahramanmaraş', 'Ankara', 'Gaziantep', 'Kütahya', 'Hakkari',
       'Eskişehir', 'Yurt dışı', 'Ağrı', 'Osmaniye', 'Sakarya', 'Tokat',
       'Antalya', 'Erzurum', 'Batman', 'Bilecik', 'Kayseri', 'Adana',
       'Malatya', 'Sivas', 'Denizli', 'Şanlıurfa', 'Diyarbakır',
       'Zonguldak', 'Kocaeli', 'İzmir', 'Bingöl', 'Van', 'Bitlis',
       'Siirt', 'Aksaray', 'Balıkesir', 'Şırnak', 'Uşak', 'Tunceli',
       'Muğla', 'Aydın', 'Muş', 'Karaman', 'Çorum', 'Kırşehir',
       'Kırıkkale', 'Çanakkale', 'Elazığ', 'Giresun', 'Kars', 'Edirne',
       'Çankırı', 'Kastamonu', 'KKTC', 'Afyonkarahisar', 'Manisa',
       'Yozgat', 'Trabzon', 'Burdur', 'Niğde', 'Ardahan', 'Bolu',
       'Erzincan', 'Artvin', 'Kırklareli', 'Karabük', 'Iğdır', 'Nevşehir',
       'Rize', 'Kilis', 'Düzce', 'Sinop', 'Bayburt', 'Gümüşhane',
       'Bartın', 'Yalova'], dtype=object)

In [23]:
train_data["dogum_yeri"].value_counts()

dogum_yeri
İstanbul      1814
Ankara         603
Diyarbakır     408
Hatay          384
Şanlıurfa      357
              ... 
Bartın          14
Yalova          14
KKTC            10
Tunceli          9
Bayburt          9
Name: count, Length: 83, dtype: int64

In [24]:
turkish_cities = {}

with open("datathon-dataset/processed-data/turkish_cities.json") as json_file:
    data = json.load(json_file)
    turkish_cities = {int(key): value for key, value in data.items()}
    print(turkish_cities)

{1: 'Adana', 2: 'Adıyaman', 3: 'Afyonkarahisar', 4: 'Ağrı', 5: 'Amasya', 6: 'Ankara', 7: 'Antalya', 8: 'Artvin', 9: 'Aydın', 10: 'Balıkesir', 11: 'Bilecik', 12: 'Bingöl', 13: 'Bitlis', 14: 'Bolu', 15: 'Burdur', 16: 'Bursa', 17: 'Çanakkale', 18: 'Çankırı', 19: 'Çorum', 20: 'Denizli', 21: 'Diyarbakır', 22: 'Edirne', 23: 'Elazığ', 24: 'Erzincan', 25: 'Erzurum', 26: 'Eskişehir', 27: 'Gaziantep', 28: 'Giresun', 29: 'Gümüşhane', 30: 'Hakkâri', 31: 'Hatay', 32: 'Isparta', 33: 'Mersin', 34: 'İstanbul', 35: 'İzmir', 36: 'Kars', 37: 'Kastamonu', 38: 'Kayseri', 39: 'Kırklareli', 40: 'Kırşehir', 41: 'Kocaeli', 42: 'Konya', 43: 'Kütahya', 44: 'Malatya', 45: 'Manisa', 46: 'Kahramanmaraş', 47: 'Mardin', 48: 'Muğla', 49: 'Muş', 50: 'Nevşehir', 51: 'Niğde', 52: 'Ordu', 53: 'Rize', 54: 'Sakarya', 55: 'Samsun', 56: 'Siirt', 57: 'Sinop', 58: 'Sivas', 59: 'Tekirdağ', 60: 'Tokat', 61: 'Trabzon', 62: 'Tunceli', 63: 'Şanlıurfa', 64: 'Uşak', 65: 'Van', 66: 'Yozgat', 67: 'Zonguldak', 68: 'Aksaray', 69: 'Bayburt

In [25]:
def clean_city_name(city):
    if pd.isna(city) or not isinstance(city, str):
        return pd.NA

    # Convert to lowercase and remove leading/trailing whitespace
    city = city.lower().strip()

    # Remove any digits
    city = re.sub(r"\d+", "", city)

    # Remove any parentheses and their contents
    city = re.sub(r"\([^)]*\)", "", city)

    # Remove common words
    remove_words = ["şehir", "ili", "ilçesi", "merkez"]
    for word in remove_words:
        city = city.replace(word, "")

    # Remove any remaining punctuation and extra whitespace
    city = re.sub(r"[^\w\s]", "", city)
    city = " ".join(city.split())

    # Use fuzzy matching to find the closest match
    city_list = list(turkish_cities.values())
    closest_match, score = process.extractOne(city, city_list)

    if score >= 80:
        for key, value in turkish_cities.items():
            if value == closest_match:
                return key
    else:
        return pd.NA

In [26]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,Amasya,Amasya,Munzur Üniversitesi,Devlet,NaN,Hayır,Sosyal Hizmet,1,...,2.0,Hayır,Evet,Evet,NaN,Evet,NaN,0,2002,6
1,2023,1,Konya,Konya,Hacettepe Üniversitesi,Devlet,NaN,Hayır,Uluslararası İlişkiler,1,...,3.0,Hayır,Evet,Hayır,NaN,Evet,NaN,1,2004,5
2,2023,0,İstanbul,İstanbul,Kapadokya Üniversitesi,Özel,NaN,Evet,Diğer,1,...,0.0,Hayır,Hayır,Hayır,NaN,Evet,NaN,2,2002,4
3,2023,0,Mardin,Mardin,Mardin Artuklu Üniversitesi,Devlet,NaN,Hayır,İslami İlimler,1,...,4.0,Hayır,Hayır,Evet,NaN,Hayır,NaN,3,2003,3
4,2023,0,Samsun,İstanbul,Boğaziçi Üniversitesi,Devlet,NaN,Hayır,İşletme,3,...,1.0,Hayır,Hayır,Evet,NaN,Evet,NaN,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,Yozgat,Ankara,Gazi Üniversitesi,Devlet,NaN,Hayır,Endüstri Mühendisliği,2,...,0.0,Evet,Hayır,Evet,NaN,Evet,NaN,11044,2002,1
11045,2023,1,Konya,Konya,Niğde Ömer Halisdemir Üniversitesi,Devlet,NaN,Hayır,Gıda Mühendisliği,3,...,3.0,Evet,Hayır,Evet,NaN,Hayır,NaN,11045,2001,9
11046,2023,0,Ankara,Ankara,Karabük Üniversitesi,Devlet,NaN,Hayır,Elektrik-Elektronik Mühendisliği,1,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11046,2004,6
11047,2023,1,Adıyaman,İstanbul,Bursa Uludağ Üniversitesi,Devlet,NaN,Hayır,"Moleküler Biyoloji, Genetik ve Biyomühendislik",4,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11047,2001,10


In [27]:
train_data["dogum_yeri"] = train_data["dogum_yeri"].apply(clean_city_name)

In [28]:
train_data["dogum_yeri"].unique()

array([5, 42, 34, 47, 55, 32, 31, 52, 2, 16, 59, 33, 46, 6, 27, 43, 30,
       26, <NA>, 4, 80, 54, 60, 7, 25, 72, 11, 38, 1, 44, 58, 20, 63, 21,
       67, 41, 35, 12, 65, 13, 56, 68, 10, 73, 64, 62, 48, 9, 49, 70, 19,
       18, 71, 17, 23, 28, 36, 22, 37, 3, 45, 66, 61, 15, 51, 75, 14, 24,
       8, 39, 78, 76, 50, 53, 81, 57, 69, 29, 74, 77], dtype=object)

In [29]:
train_data["ikametgah_sehri"]

0          Amasya
1           Konya
2        İstanbul
3          Mardin
4        İstanbul
           ...   
11044      Ankara
11045       Konya
11046      Ankara
11047    İstanbul
11048      Ankara
Name: ikametgah_sehri, Length: 11049, dtype: object

In [30]:
train_data["ikametgah_sehri"] = train_data["ikametgah_sehri"].apply(clean_city_name)

In [31]:
train_data["dogum_yeri"] = train_data.apply(
    lambda row: (
        row["ikametgah_sehri"]
        if pd.isnull(row["dogum_yeri"]) or row["dogum_yeri"] == ""
        else row["dogum_yeri"]
    ),
    axis=1,
)

In [32]:
train_data[train_data["dogum_yeri"].isna()]

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
226,2023,1,<NA>,<NA>,Orta Doğu Teknik Üniversitesi,Devlet,NaN,Hayır,Fizik,1,...,2.0,Hayır,Hayır,Evet,NaN,Evet,NaN,226,2001,9
2186,2023,1,<NA>,<NA>,MEF Üniversitesi,Özel,NaN,Evet,Psikoloji,1,...,2.0,Hayır,Hayır,Evet,NaN,Evet,NaN,2186,2003,9
4440,2023,0,<NA>,<NA>,Sağlık Bilimleri Üniversitesi,Devlet,NaN,Hayır,Diş Hekimliği,2,...,4.0,Hayır,Hayır,Evet,NaN,Evet,NaN,4440,2000,8
5197,2023,1,<NA>,<NA>,Yıldız Teknik Üniversitesi,Devlet,NaN,Hayır,İşletme,4,...,2.0,Hayır,Hayır,Hayır,NaN,Evet,NaN,5197,2002,8
5612,2023,0,<NA>,<NA>,Sakarya Üniversitesi,Devlet,NaN,Hayır,Makine Mühendisliği,1,...,1.0,Hayır,Hayır,Hayır,NaN,Evet,NaN,5612,2005,1
7426,2023,1,<NA>,<NA>,Orta Doğu Teknik Üniversitesi,Devlet,NaN,Hayır,İşletme,4,...,1.0,Hayır,Hayır,Evet,NaN,Evet,NaN,7426,1999,2
10646,2023,1,<NA>,<NA>,Yurtdışında üniversite okuyorum.,Devlet,NaN,Hayır,Muhasebe ve Denetim,5,...,1.0,Evet,Evet,Evet,NaN,Evet,NaN,10646,2001,12


In [33]:
train_data["ikametgah_sehri"] = train_data.apply(
    lambda row: (
        row["dogum_yeri"]
        if pd.isnull(row["ikametgah_sehri"]) or row["ikametgah_sehri"] == ""
        else row["ikametgah_sehri"]
    ),
    axis=1,
)

In [34]:
train_data["dogum_yeri"].fillna(train_data["dogum_yeri"].mode()[0], inplace=True)
train_data["ikametgah_sehri"].fillna(
    train_data["ikametgah_sehri"].mode()[0], inplace=True
)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16198/4291801807.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["dogum_yeri"].fillna(train_data["dogum_yeri"].mode()[0], inplace=True)
/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16198/4291801807.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_sile

In [35]:
train_data[train_data["ikametgah_sehri"].isna()]

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi


In [36]:
train_data[train_data["dogum_yeri"].isna()]

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi


In [37]:
train_data["dogum_yeri"].value_counts(dropna=False)

dogum_yeri
34    1868
6      619
31     445
21     409
63     364
      ... 
11      15
74      14
77      14
62       9
69       9
Name: count, Length: 79, dtype: int64

In [38]:
train_data["dogum_yeri"].unique()

array([ 5, 42, 34, 47, 55, 32, 31, 52,  2, 16, 59, 33, 46,  6, 27, 43, 30,
       26,  4, 80, 54, 60,  7, 25, 72, 11, 38,  1, 44, 58, 20, 63, 21, 67,
       41, 35, 12, 65, 13, 56, 68, 10, 73, 64, 62, 48,  9, 49, 70, 19, 18,
       71, 17, 23, 28, 36, 22, 37,  3, 45, 66, 61, 15, 51, 75, 14, 24,  8,
       39, 78, 76, 50, 53, 81, 57, 69, 29, 74, 77])

In [39]:
train_data["ikametgah_sehri"].unique()

array([ 5, 42, 34, 47, 35, 18,  2, 16,  7,  6, 59, 27, 43, 30, 31, 60, 45,
       80, 28, 48, 25, 46, 72, 38, 17,  1, 44, 58, 63, 21, 62, 53, 33, 12,
       56, 68, 73, 23, 65,  9,  4, 49, 19, 15, 41, 10, 55, 64, 22, 52, 26,
       37, 78, 36, 13, 54,  3, 67, 32, 61, 51, 77,  8, 70, 20, 69, 24, 11,
       75, 50, 66, 14, 81, 76, 71, 74, 29, 57, 39])

In [40]:
train_data["universite_adi"].unique()

array(['Munzur Üniversitesi', 'Hacettepe Üniversitesi',
       'Kapadokya Üniversitesi', 'Mardin Artuklu Üniversitesi',
       'Boğaziçi Üniversitesi', 'İzmir Yüksek Teknoloji Enstitüsü',
       'Kırşehir Ahi Evran Üniversitesi', 'Yıldız Teknik Üniversitesi',
       'Necmettin Erbakan Üniversitesi', 'Bursa Uludağ Üniversitesi',
       'MEF Üniversitesi', 'Eskişehir Teknik Üniversitesi',
       'İstanbul Üniversitesi-Cerrahpaşa', 'Gazi Üniversitesi',
       'Tekirdağ Namık Kemal Üniversitesi', 'Gaziantep Üniversitesi',
       'Koç Üniversitesi', 'Ankara Hacı Bayram Veli Üniversitesi',
       'Çukurova Üniversitesi', 'Kütahya Dumlupınar Üniversitesi',
       'Ankara Bilim Üniversitesi', 'Dicle Üniversitesi',
       'Dokuz Eylül Üniversitesi', 'Siirt Üniversitesi',
       'Kilis 7 Aralık Üniversitesi',
       'Çanakkale Onsekiz Mart Üniversitesi',
       'Sivas Cumhuriyet Üniversitesi',
       'Tokat Gaziosmanpaşa Üniversitesi', 'Nişantaşı Üniversitesi',
       'Özyeğin Üniversitesi', 'Ma

In [41]:
universities = pd.read_html(
    "https://tr.wikipedia.org/wiki/T%C3%BCrkiye%27deki_%C3%BCniversiteler_listesi"
)[0]["Üniversite"].to_dict()

In [42]:
def clean_university_name(university):
    if pd.isna(university):
        return pd.NA

    # Convert to lowercase and remove leading/trailing whitespace
    university = university.lower().strip()

    # Remove any digits
    university = re.sub(r"\d+", "", university)

    # Remove any parentheses and their contents
    university = re.sub(r"\([^)]*\)", "", university)

    # Remove any remaining punctuation and extra whitespace
    university = re.sub(r"[^\w\s]", "", university)
    university = " ".join(university.split())

    # Use fuzzy matching to find the closest match
    university_list = list(universities.values())
    closest_match, score = process.extractOne(university, university_list)

    if score >= 80:
        for key, value in universities.items():
            if value == closest_match:
                return key
    else:
        return university

In [43]:
train_data["universite_adi"] = train_data["universite_adi"].apply(clean_university_name)

In [44]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199,Devlet,NaN,Hayır,Sosyal Hizmet,1,...,2.0,Hayır,Evet,Evet,NaN,Evet,NaN,0,2002,6
1,2023,1,42,42,14,Devlet,NaN,Hayır,Uluslararası İlişkiler,1,...,3.0,Hayır,Evet,Hayır,NaN,Evet,NaN,1,2004,5
2,2023,0,34,34,179,Özel,NaN,Evet,Diğer,1,...,0.0,Hayır,Hayır,Hayır,NaN,Evet,NaN,2,2002,4
3,2023,0,47,47,171,Devlet,NaN,Hayır,İslami İlimler,1,...,4.0,Hayır,Hayır,Evet,NaN,Hayır,NaN,3,2003,3
4,2023,0,55,34,78,Devlet,NaN,Hayır,İşletme,3,...,1.0,Hayır,Hayır,Evet,NaN,Evet,NaN,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13,Devlet,NaN,Hayır,Endüstri Mühendisliği,2,...,0.0,Evet,Hayır,Evet,NaN,Evet,NaN,11044,2002,1
11045,2023,1,42,42,180,Devlet,NaN,Hayır,Gıda Mühendisliği,3,...,3.0,Evet,Hayır,Evet,NaN,Hayır,NaN,11045,2001,9
11046,2023,0,6,6,146,Devlet,NaN,Hayır,Elektrik-Elektronik Mühendisliği,1,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11046,2004,6
11047,2023,1,2,34,50,Devlet,NaN,Hayır,"Moleküler Biyoloji, Genetik ve Biyomühendislik",4,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11047,2001,10


In [45]:
train_data["universite_adi"].unique()

array([199, 14, 179, 171, 78, 137, 156, 90, 162, 50, 129, 65, 83, 13, 194,
       66, 127, 11, 1, 166, 18, 56, 135, 188, 157, 52, 190, 195, 117, 130,
       169, 176, 31, 168, 82, 15, 70, 80, 132, 59, 42, 107, 150, 81, 96,
       61, 187, 63, 106, 172, 98, 108, 116, 151, 186, 37, 2, 158, 73, 125,
       148, 40, 149, 192, 39, 16, 'diğer', 43, 93, 163, 84, 144, 89, 105,
       183, 71, 126, 60, 128, 27, 138, 181, 57, 9, 48, 161, 170, 184, 118,
       38, 23, 113, 134, 45, 7, 160, 22, 146, 196, 200, 178, 24, 55, 58,
       140, 4, 197, 6, 28, 141, 32, 74, 79, 0, 67, 49, 112, 76, 159, 115,
       139, 136, 182, 201, 86, 47, 68, 3, 109, 26, 114, 180, 77, 204, 35,
       177, 64, 202, 101, 147, 185, 46, 104, 44, 29, 54, 155, 124, 92, 72,
       87, 5, 131, 164, 167, 'kktcde üniversite okuyorum', 133, 95, 189,
       193, 12, 75, 174, 110, 91, 143, 21, 36, 94, 173, 53, 119, 111, 122,
       203, 'yurtdışında üniversite okuyorum', 120, 20, 62, 165, 100, 102,
       97, 154, 198, 19, 152, 99, 

In [46]:
train_data["universite_adi"] = pd.to_numeric(
    train_data["universite_adi"],
    errors="coerce",
)
train_data["universite_adi"].fillna(
    train_data["universite_adi"].mode()[0], inplace=True
)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16198/706566557.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["universite_adi"].fillna(


In [47]:
train_data["universite_adi"].value_counts(dropna=False)

universite_adi
81.0     404
90.0     316
84.0     305
78.0     277
82.0     259
        ... 
142.0      2
97.0       2
30.0       2
85.0       1
10.0       1
Name: count, Length: 199, dtype: int64

In [48]:
train_data["universite_adi"].unique()

array([199.,  14., 179., 171.,  78., 137., 156.,  90., 162.,  50., 129.,
        65.,  83.,  13., 194.,  66., 127.,  11.,   1., 166.,  18.,  56.,
       135., 188., 157.,  52., 190., 195., 117., 130., 169., 176.,  31.,
       168.,  82.,  15.,  70.,  80., 132.,  59.,  42., 107., 150.,  81.,
        96.,  61., 187.,  63., 106., 172.,  98., 108., 116., 151., 186.,
        37.,   2., 158.,  73., 125., 148.,  40., 149., 192.,  39.,  16.,
        43.,  93., 163.,  84., 144.,  89., 105., 183.,  71., 126.,  60.,
       128.,  27., 138., 181.,  57.,   9.,  48., 161., 170., 184., 118.,
        38.,  23., 113., 134.,  45.,   7., 160.,  22., 146., 196., 200.,
       178.,  24.,  55.,  58., 140.,   4., 197.,   6.,  28., 141.,  32.,
        74.,  79.,   0.,  67.,  49., 112.,  76., 159., 115., 139., 136.,
       182., 201.,  86.,  47.,  68.,   3., 109.,  26., 114., 180.,  77.,
       204.,  35., 177.,  64., 202., 101., 147., 185.,  46., 104.,  44.,
        29.,  54., 155., 124.,  92.,  72.,  87.,   

In [49]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,Devlet,NaN,Hayır,Sosyal Hizmet,1,...,2.0,Hayır,Evet,Evet,NaN,Evet,NaN,0,2002,6
1,2023,1,42,42,14.0,Devlet,NaN,Hayır,Uluslararası İlişkiler,1,...,3.0,Hayır,Evet,Hayır,NaN,Evet,NaN,1,2004,5
2,2023,0,34,34,179.0,Özel,NaN,Evet,Diğer,1,...,0.0,Hayır,Hayır,Hayır,NaN,Evet,NaN,2,2002,4
3,2023,0,47,47,171.0,Devlet,NaN,Hayır,İslami İlimler,1,...,4.0,Hayır,Hayır,Evet,NaN,Hayır,NaN,3,2003,3
4,2023,0,55,34,78.0,Devlet,NaN,Hayır,İşletme,3,...,1.0,Hayır,Hayır,Evet,NaN,Evet,NaN,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,Devlet,NaN,Hayır,Endüstri Mühendisliği,2,...,0.0,Evet,Hayır,Evet,NaN,Evet,NaN,11044,2002,1
11045,2023,1,42,42,180.0,Devlet,NaN,Hayır,Gıda Mühendisliği,3,...,3.0,Evet,Hayır,Evet,NaN,Hayır,NaN,11045,2001,9
11046,2023,0,6,6,146.0,Devlet,NaN,Hayır,Elektrik-Elektronik Mühendisliği,1,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11046,2004,6
11047,2023,1,2,34,50.0,Devlet,NaN,Hayır,"Moleküler Biyoloji, Genetik ve Biyomühendislik",4,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11047,2001,10


In [50]:
train_data["universite_adi"].isna().sum()

0

In [51]:
train_data["universite_turu"].value_counts()

universite_turu
Devlet    9215
Özel      1834
Name: count, dtype: int64

In [52]:
university_type_map = {"Devlet": 0, "DEVLET": 0, "Özel": 1, "ÖZEL": 1, "nan": -1}

In [53]:
train_data["universite_turu"] = train_data["universite_turu"].map(university_type_map)

In [54]:
train_data["universite_turu"].unique()

array([0, 1])

In [55]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,NaN,Hayır,Sosyal Hizmet,1,...,2.0,Hayır,Evet,Evet,NaN,Evet,NaN,0,2002,6
1,2023,1,42,42,14.0,0,NaN,Hayır,Uluslararası İlişkiler,1,...,3.0,Hayır,Evet,Hayır,NaN,Evet,NaN,1,2004,5
2,2023,0,34,34,179.0,1,NaN,Evet,Diğer,1,...,0.0,Hayır,Hayır,Hayır,NaN,Evet,NaN,2,2002,4
3,2023,0,47,47,171.0,0,NaN,Hayır,İslami İlimler,1,...,4.0,Hayır,Hayır,Evet,NaN,Hayır,NaN,3,2003,3
4,2023,0,55,34,78.0,0,NaN,Hayır,İşletme,3,...,1.0,Hayır,Hayır,Evet,NaN,Evet,NaN,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,NaN,Hayır,Endüstri Mühendisliği,2,...,0.0,Evet,Hayır,Evet,NaN,Evet,NaN,11044,2002,1
11045,2023,1,42,42,180.0,0,NaN,Hayır,Gıda Mühendisliği,3,...,3.0,Evet,Hayır,Evet,NaN,Hayır,NaN,11045,2001,9
11046,2023,0,6,6,146.0,0,NaN,Hayır,Elektrik-Elektronik Mühendisliği,1,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11046,2004,6
11047,2023,1,2,34,50.0,0,NaN,Hayır,"Moleküler Biyoloji, Genetik ve Biyomühendislik",4,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11047,2001,10


In [56]:
train_data["universite_turu"].value_counts(dropna=False)

universite_turu
0    9215
1    1834
Name: count, dtype: int64

In [57]:
train_data["burs_aliyor_mu?"].unique()

array(['Hayır', 'Evet'], dtype=object)

In [58]:
is_there_scholarship = {"Evet": 1, "EVET": 1, "Hayır": 0, "hayır": 0}
train_data["burs_aliyor_mu?"] = train_data["burs_aliyor_mu?"].map(is_there_scholarship)

In [59]:
train_data["burs_aliyor_mu?"] = train_data["burs_aliyor_mu?"].fillna(0)

In [60]:
train_data["burs_aliyor_mu?"].value_counts(dropna=False)

burs_aliyor_mu?
0    9400
1    1649
Name: count, dtype: int64

In [61]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,NaN,0,Sosyal Hizmet,1,...,2.0,Hayır,Evet,Evet,NaN,Evet,NaN,0,2002,6
1,2023,1,42,42,14.0,0,NaN,0,Uluslararası İlişkiler,1,...,3.0,Hayır,Evet,Hayır,NaN,Evet,NaN,1,2004,5
2,2023,0,34,34,179.0,1,NaN,1,Diğer,1,...,0.0,Hayır,Hayır,Hayır,NaN,Evet,NaN,2,2002,4
3,2023,0,47,47,171.0,0,NaN,0,İslami İlimler,1,...,4.0,Hayır,Hayır,Evet,NaN,Hayır,NaN,3,2003,3
4,2023,0,55,34,78.0,0,NaN,0,İşletme,3,...,1.0,Hayır,Hayır,Evet,NaN,Evet,NaN,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,NaN,0,Endüstri Mühendisliği,2,...,0.0,Evet,Hayır,Evet,NaN,Evet,NaN,11044,2002,1
11045,2023,1,42,42,180.0,0,NaN,0,Gıda Mühendisliği,3,...,3.0,Evet,Hayır,Evet,NaN,Hayır,NaN,11045,2001,9
11046,2023,0,6,6,146.0,0,NaN,0,Elektrik-Elektronik Mühendisliği,1,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11046,2004,6
11047,2023,1,2,34,50.0,0,NaN,0,"Moleküler Biyoloji, Genetik ve Biyomühendislik",4,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11047,2001,10


In [62]:
train_data["burslu_ise_burs_yuzdesi"].unique()

array([nan])

In [63]:
train_data["burslu_ise_burs_yuzdesi"] = train_data.apply(
    lambda row: 100 if row["burs_aliyor_mu?"] == 1 else 0, axis=1
)

In [64]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,Sosyal Hizmet,1,...,2.0,Hayır,Evet,Evet,NaN,Evet,NaN,0,2002,6
1,2023,1,42,42,14.0,0,0,0,Uluslararası İlişkiler,1,...,3.0,Hayır,Evet,Hayır,NaN,Evet,NaN,1,2004,5
2,2023,0,34,34,179.0,1,100,1,Diğer,1,...,0.0,Hayır,Hayır,Hayır,NaN,Evet,NaN,2,2002,4
3,2023,0,47,47,171.0,0,0,0,İslami İlimler,1,...,4.0,Hayır,Hayır,Evet,NaN,Hayır,NaN,3,2003,3
4,2023,0,55,34,78.0,0,0,0,İşletme,3,...,1.0,Hayır,Hayır,Evet,NaN,Evet,NaN,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,Endüstri Mühendisliği,2,...,0.0,Evet,Hayır,Evet,NaN,Evet,NaN,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,Gıda Mühendisliği,3,...,3.0,Evet,Hayır,Evet,NaN,Hayır,NaN,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,Elektrik-Elektronik Mühendisliği,1,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,"Moleküler Biyoloji, Genetik ve Biyomühendislik",4,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11047,2001,10


In [65]:
train_data["burslu_ise_burs_yuzdesi"].value_counts(dropna=False)

burslu_ise_burs_yuzdesi
0      9400
100    1649
Name: count, dtype: int64

In [66]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,Sosyal Hizmet,1,...,2.0,Hayır,Evet,Evet,NaN,Evet,NaN,0,2002,6
1,2023,1,42,42,14.0,0,0,0,Uluslararası İlişkiler,1,...,3.0,Hayır,Evet,Hayır,NaN,Evet,NaN,1,2004,5
2,2023,0,34,34,179.0,1,100,1,Diğer,1,...,0.0,Hayır,Hayır,Hayır,NaN,Evet,NaN,2,2002,4
3,2023,0,47,47,171.0,0,0,0,İslami İlimler,1,...,4.0,Hayır,Hayır,Evet,NaN,Hayır,NaN,3,2003,3
4,2023,0,55,34,78.0,0,0,0,İşletme,3,...,1.0,Hayır,Hayır,Evet,NaN,Evet,NaN,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,Endüstri Mühendisliği,2,...,0.0,Evet,Hayır,Evet,NaN,Evet,NaN,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,Gıda Mühendisliği,3,...,3.0,Evet,Hayır,Evet,NaN,Hayır,NaN,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,Elektrik-Elektronik Mühendisliği,1,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,"Moleküler Biyoloji, Genetik ve Biyomühendislik",4,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11047,2001,10


In [67]:
train_data.columns

Index(['basvuru_yili', 'cinsiyet', 'dogum_yeri', 'ikametgah_sehri',
       'universite_adi', 'universite_turu', 'burslu_ise_burs_yuzdesi',
       'burs_aliyor_mu?', 'bölüm', 'universite_kacinci_sinif',
       'universite_not_ortalamasi', 'lise_adi', 'lise_sehir',
       'lise_mezuniyet_notu', 'anne_egitim_durumu', 'anne_calisma_durumu',
       'anne_sektor', 'baba_egitim_durumu', 'baba_calisma_durumu',
       'baba_sektor', 'kardes_sayisi',
       'girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?',
       'profesyonel_bir_spor_daliyla_mesgul_musunuz?',
       'aktif_olarak_bir_stk_üyesi_misiniz?', 'stk_projesine_katildiniz_mi?',
       'ingilizce_biliyor_musunuz?', 'ingilizce_seviyeniz?', 'id',
       'dogum_yili', 'dogum_ayi'],
      dtype='object')

In [68]:
train_data["bölüm"].value_counts()

bölüm
Bilgisayar Mühendisliği               817
Endüstri Mühendisliği                 635
Tıp                                   456
İşletme                               425
Hukuk                                 383
                                     ... 
Japonca Öğretmenliği                    1
İstatistik ve Bilgisayar Bilimleri      1
Film Tasarımı ve Yazarlığı              1
İşletme Bilgi Yönetimi                  1
İspanyol Dili ve Edebiyatı              1
Name: count, Length: 349, dtype: int64

In [69]:
bolumler = pd.read_html("https://hangiuniversite.com/bolumler")[0]
bolumler = pd.DataFrame(bolumler)
majors = bolumler.apply(lambda x: x.str.lstrip("» ")).to_dict()["Bölüm"]

In [70]:
majors

{0: 'ACİL YARDIM VE AFET YÖNETİMİ(FAKÜLTE)',
 1: 'ACİL YARDIM VE AFET YÖNETİMİ(YÜKSEKOKUL)',
 2: 'ADLİ BİLİMLER',
 3: 'ADLİ BİLİŞİM MÜHENDİSLİĞİ',
 4: 'ADLİ BİLİŞİM MÜHENDİSLİĞİ(M.T.O.K.)',
 5: 'AĞAÇ İŞLERİ ENDÜSTRİ MÜHENDİSLİĞİ',
 6: 'AĞAÇ İŞLERİ ENDÜSTRİ MÜHENDİSLİĞİ(M.T.O.K.)',
 7: 'AİLE VE TÜKETİCİ BİLİMLERİ',
 8: 'AKTÜERYA BİLİMLERİ(FAKÜLTE)',
 9: 'ALMAN DİLİ VE EDEBİYATI',
 10: 'ALMANCA MÜTERCİM VE TERCÜMANLIK(FAKÜLTE)',
 11: 'ALMANCA MÜTERCİM VE TERCÜMANLIK(YÜKSEKOKUL)',
 12: 'ALMANCA ÖĞRETMENLİĞİ',
 13: 'AMERİKAN KÜLTÜRÜ VE EDEBİYATI',
 14: 'ANTRENÖRLÜK EĞİTİMİ(FAKÜLTE)',
 15: 'ANTRENÖRLÜK EĞİTİMİ(YÜKSEKOKUL)',
 16: 'ANTROPOLOJİ',
 17: 'ARAP DİLİ VE EDEBİYATI',
 18: 'ARAPÇA MÜTERCİM VE TERCÜMANLIK(FAKÜLTE)',
 19: 'ARAPÇA MÜTERCİM VE TERCÜMANLIK(YÜKSEKOKUL)',
 20: 'ARAPÇA ÖĞRETMENLİĞİ',
 21: 'ARKEOLOJİ',
 22: 'ARKEOLOJİ VE SANAT TARİHİ',
 23: 'ARNAVUT DİLİ VE EDEBİYATI',
 24: 'ASTRONOMİ VE UZAY BİLİMLERİ',
 25: 'AYAKKABI TASARIMI VE ÜRETİMİ',
 26: 'AZERBAYCAN TÜRKÇESİ VE EDEBİYA

In [71]:
def clean_major_name(major):
    if pd.isna(major):
        return pd.NA

    # Convert to lowercase and remove leading/trailing whitespace
    major = major.lower().strip()

    # Remove any digits
    major = re.sub(r"\d+", "", major)

    # Remove any parentheses and their contents
    major = re.sub(r"\([^)]*\)", "", major)

    # Remove any remaining punctuation and extra whitespace
    major = re.sub(r"[^\w\s]", "", major)
    major = " ".join(major.split())

    # Use fuzzy matching to find the closest match
    majors_list = list(majors.values())
    closest_match, score = process.extractOne(major, majors_list)

    if score >= 80:
        for key, value in majors.items():
            if value == closest_match:
                return key
    else:
        return major

In [72]:
train_data["bölüm"] = train_data["bölüm"].apply(clean_major_name)

In [73]:
train_data["bölüm"]

0          347
1          432
2        diğer
3          228
4          230
         ...  
11044      111
11045      159
11046      104
11047        9
11048      111
Name: bölüm, Length: 11049, dtype: object

In [74]:
train_data["bölüm"] = pd.to_numeric(train_data["bölüm"], errors="coerce")
train_data["bölüm"].fillna(train_data["bölüm"].mode()[0], inplace=True)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16198/3012943815.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["bölüm"].fillna(train_data["bölüm"].mode()[0], inplace=True)


In [75]:
train_data["bölüm"].value_counts(dropna=False)

bölüm
44.0     2194
111.0     635
230.0     425
194.0     383
189.0     369
         ... 
450.0       1
122.0       1
107.0       1
239.0       1
233.0       1
Name: count, Length: 235, dtype: int64

In [76]:
train_data["bölüm"].unique()

array([347., 432.,  44., 228., 230., 189.,  77., 203.,  12., 210., 282.,
       111.,  53., 194., 274., 201., 104., 323.,  34., 312.,  60.,  30.,
       328., 302., 191.,  49., 289., 454., 219., 371.,  91.,   0., 427.,
         1., 109., 414.,  58., 348., 136., 415., 267., 112., 128., 367.,
       363., 291.,  36., 275., 159., 435.,  14., 365., 135.,  87., 240.,
       280., 253., 450.,  73., 300., 231., 217., 248., 313., 134., 250.,
        99., 249., 117.,  56., 146., 384., 362., 447.,  47., 345., 165.,
       299., 163., 316., 326., 207., 103.,  68., 285., 106., 449., 234.,
       247., 278., 155., 269.,   3., 408., 151., 276., 419., 332., 211.,
       307., 179., 389.,  80.,   2., 295., 125., 197.,  17., 271., 351.,
       157.,  76., 172.,  27., 358., 170., 158., 142., 330., 359., 100.,
       184., 221., 277., 453.,  55., 242.,   8., 149.,  48., 298., 124.,
       388.,   5., 143., 144., 290.,  43., 434.,  85., 181., 423., 257.,
       459.,  40., 394., 145., 266.,  33.,   9.,  4

In [77]:
train_data["universite_kacinci_sinif"].unique()

array(['1', '3', '2', '4', 'Hazırlık', '5', '6'], dtype=object)

In [78]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,347.0,1,...,2.0,Hayır,Evet,Evet,NaN,Evet,NaN,0,2002,6
1,2023,1,42,42,14.0,0,0,0,432.0,1,...,3.0,Hayır,Evet,Hayır,NaN,Evet,NaN,1,2004,5
2,2023,0,34,34,179.0,1,100,1,44.0,1,...,0.0,Hayır,Hayır,Hayır,NaN,Evet,NaN,2,2002,4
3,2023,0,47,47,171.0,0,0,0,228.0,1,...,4.0,Hayır,Hayır,Evet,NaN,Hayır,NaN,3,2003,3
4,2023,0,55,34,78.0,0,0,0,230.0,3,...,1.0,Hayır,Hayır,Evet,NaN,Evet,NaN,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,111.0,2,...,0.0,Evet,Hayır,Evet,NaN,Evet,NaN,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,159.0,3,...,3.0,Evet,Hayır,Evet,NaN,Hayır,NaN,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,104.0,1,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,9.0,4,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11047,2001,10


In [79]:
university_grade_map = {
    "nan": 0,
    "0": 0,
    "1": 1,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "Mezun": 7,
    "Yüksek Lisans": 8,
    "Tez": 9,
    "hazırlık": 10,
}

In [80]:
train_data["universite_kacinci_sinif"] = pd.to_numeric(
    train_data["universite_kacinci_sinif"].map(university_grade_map),
    errors="coerce",
)
train_data["universite_kacinci_sinif"] = train_data["universite_kacinci_sinif"].fillna(
    0
)

In [81]:
train_data["universite_kacinci_sinif"].unique()

array([1., 3., 2., 4., 0., 5., 6.])

In [82]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,347.0,1.0,...,2.0,Hayır,Evet,Evet,NaN,Evet,NaN,0,2002,6
1,2023,1,42,42,14.0,0,0,0,432.0,1.0,...,3.0,Hayır,Evet,Hayır,NaN,Evet,NaN,1,2004,5
2,2023,0,34,34,179.0,1,100,1,44.0,1.0,...,0.0,Hayır,Hayır,Hayır,NaN,Evet,NaN,2,2002,4
3,2023,0,47,47,171.0,0,0,0,228.0,1.0,...,4.0,Hayır,Hayır,Evet,NaN,Hayır,NaN,3,2003,3
4,2023,0,55,34,78.0,0,0,0,230.0,3.0,...,1.0,Hayır,Hayır,Evet,NaN,Evet,NaN,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,111.0,2.0,...,0.0,Evet,Hayır,Evet,NaN,Evet,NaN,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,159.0,3.0,...,3.0,Evet,Hayır,Evet,NaN,Hayır,NaN,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,104.0,1.0,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,9.0,4.0,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11047,2001,10


In [83]:
train_data["universite_kacinci_sinif"].value_counts(dropna=False)

universite_kacinci_sinif
2.0    3315
3.0    3039
4.0    2466
1.0    1803
0.0     233
5.0     164
6.0      29
Name: count, dtype: int64

In [84]:
train_data["universite_not_ortalamasi"].unique()

array(['2.50 - 2.99', 'Hazırlığım', '3.00 - 3.49', '1.80 - 2.49',
       '3.50 - 4.00', '0 - 1.79'], dtype=object)

In [85]:
train_data["universite_not_ortalamasi"].value_counts()

universite_not_ortalamasi
2.50 - 2.99    3605
3.00 - 3.49    3296
1.80 - 2.49    1742
Hazırlığım     1108
3.50 - 4.00    1007
0 - 1.79        291
Name: count, dtype: int64

In [86]:
train_data["universite_not_ortalamasi"].unique()

array(['2.50 - 2.99', 'Hazırlığım', '3.00 - 3.49', '1.80 - 2.49',
       '3.50 - 4.00', '0 - 1.79'], dtype=object)

In [87]:
university_score_map = {
    "nan": "nan",
    "ORTALAMA BULUNMUYOR": "nan",
    "Hazırliğım": "nan",
    "Not Ortalaması yok": "nan",
    "Ortalama bulunmuyor": "nan",
    "2.50 ve altı": 1,
    "2.00 - 2.50": 1,
    "1.00 - 2.50": 1,
    "1.80 - 2.49": 1,
    "1 - 1.79": 1,
    "2.50 - 3.00": 2,
    "2.50 - 2.99": 2,
    "3.00-2.50": 2,
    "2.50 - 3.00": 2,
    "3.50-3": 3,
    "3.50 - 4.00": 3,
    "3.00 - 3.50": 3,
    "3.00 - 3.49": 3,
    "3.00 - 4.00": 3,
    "4-3.5": 4,
}

In [88]:
train_data["universite_not_ortalamasi"] = pd.to_numeric(
    train_data["universite_not_ortalamasi"].map(university_score_map),
    errors="coerce",
)

In [89]:
train_data["universite_not_ortalamasi"].fillna(0, inplace=True)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16198/3217699816.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["universite_not_ortalamasi"].fillna(0, inplace=True)


In [90]:
train_data["universite_not_ortalamasi"].unique()

array([2., 0., 3., 1.])

In [91]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,347.0,1.0,...,2.0,Hayır,Evet,Evet,NaN,Evet,NaN,0,2002,6
1,2023,1,42,42,14.0,0,0,0,432.0,1.0,...,3.0,Hayır,Evet,Hayır,NaN,Evet,NaN,1,2004,5
2,2023,0,34,34,179.0,1,100,1,44.0,1.0,...,0.0,Hayır,Hayır,Hayır,NaN,Evet,NaN,2,2002,4
3,2023,0,47,47,171.0,0,0,0,228.0,1.0,...,4.0,Hayır,Hayır,Evet,NaN,Hayır,NaN,3,2003,3
4,2023,0,55,34,78.0,0,0,0,230.0,3.0,...,1.0,Hayır,Hayır,Evet,NaN,Evet,NaN,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,111.0,2.0,...,0.0,Evet,Hayır,Evet,NaN,Evet,NaN,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,159.0,3.0,...,3.0,Evet,Hayır,Evet,NaN,Hayır,NaN,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,104.0,1.0,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,9.0,4.0,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11047,2001,10


In [92]:
train_data["universite_not_ortalamasi"].value_counts(dropna=False)

universite_not_ortalamasi
3.0    4303
2.0    3605
1.0    1742
0.0    1399
Name: count, dtype: int64

In [93]:
train_data["ingilizce_biliyor_musunuz?"].value_counts()

ingilizce_biliyor_musunuz?
Evet     8863
Hayır    2186
Name: count, dtype: int64

In [94]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,347.0,1.0,...,2.0,Hayır,Evet,Evet,NaN,Evet,NaN,0,2002,6
1,2023,1,42,42,14.0,0,0,0,432.0,1.0,...,3.0,Hayır,Evet,Hayır,NaN,Evet,NaN,1,2004,5
2,2023,0,34,34,179.0,1,100,1,44.0,1.0,...,0.0,Hayır,Hayır,Hayır,NaN,Evet,NaN,2,2002,4
3,2023,0,47,47,171.0,0,0,0,228.0,1.0,...,4.0,Hayır,Hayır,Evet,NaN,Hayır,NaN,3,2003,3
4,2023,0,55,34,78.0,0,0,0,230.0,3.0,...,1.0,Hayır,Hayır,Evet,NaN,Evet,NaN,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,111.0,2.0,...,0.0,Evet,Hayır,Evet,NaN,Evet,NaN,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,159.0,3.0,...,3.0,Evet,Hayır,Evet,NaN,Hayır,NaN,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,104.0,1.0,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,9.0,4.0,...,2.0,Evet,Evet,Evet,NaN,Evet,NaN,11047,2001,10


In [95]:
do_you_know_english_map = {"Evet": 1, "Hayır": 0, "nan": "nan"}

In [96]:
train_data["ingilizce_biliyor_musunuz?"] = pd.to_numeric(
    train_data["ingilizce_biliyor_musunuz?"].map(do_you_know_english_map),
    errors="coerce",
)

In [97]:
train_data["ingilizce_biliyor_musunuz?"].fillna(0, inplace=True)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16198/1022056692.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["ingilizce_biliyor_musunuz?"].fillna(0, inplace=True)


In [98]:
train_data["ingilizce_biliyor_musunuz?"].unique()

array([1, 0])

In [99]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,347.0,1.0,...,2.0,Hayır,Evet,Evet,NaN,1,NaN,0,2002,6
1,2023,1,42,42,14.0,0,0,0,432.0,1.0,...,3.0,Hayır,Evet,Hayır,NaN,1,NaN,1,2004,5
2,2023,0,34,34,179.0,1,100,1,44.0,1.0,...,0.0,Hayır,Hayır,Hayır,NaN,1,NaN,2,2002,4
3,2023,0,47,47,171.0,0,0,0,228.0,1.0,...,4.0,Hayır,Hayır,Evet,NaN,0,NaN,3,2003,3
4,2023,0,55,34,78.0,0,0,0,230.0,3.0,...,1.0,Hayır,Hayır,Evet,NaN,1,NaN,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,111.0,2.0,...,0.0,Evet,Hayır,Evet,NaN,1,NaN,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,159.0,3.0,...,3.0,Evet,Hayır,Evet,NaN,0,NaN,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,104.0,1.0,...,2.0,Evet,Evet,Evet,NaN,1,NaN,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,9.0,4.0,...,2.0,Evet,Evet,Evet,NaN,1,NaN,11047,2001,10


In [100]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,347.0,1.0,...,2.0,Hayır,Evet,Evet,NaN,1,NaN,0,2002,6
1,2023,1,42,42,14.0,0,0,0,432.0,1.0,...,3.0,Hayır,Evet,Hayır,NaN,1,NaN,1,2004,5
2,2023,0,34,34,179.0,1,100,1,44.0,1.0,...,0.0,Hayır,Hayır,Hayır,NaN,1,NaN,2,2002,4
3,2023,0,47,47,171.0,0,0,0,228.0,1.0,...,4.0,Hayır,Hayır,Evet,NaN,0,NaN,3,2003,3
4,2023,0,55,34,78.0,0,0,0,230.0,3.0,...,1.0,Hayır,Hayır,Evet,NaN,1,NaN,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,111.0,2.0,...,0.0,Evet,Hayır,Evet,NaN,1,NaN,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,159.0,3.0,...,3.0,Evet,Hayır,Evet,NaN,0,NaN,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,104.0,1.0,...,2.0,Evet,Evet,Evet,NaN,1,NaN,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,9.0,4.0,...,2.0,Evet,Evet,Evet,NaN,1,NaN,11047,2001,10


In [101]:
def english_level_cleaner(df: pd.DataFrame):
    df.loc[
        df["ingilizce_biliyor_musunuz?"].isin(["nan", "0", 0]), "ingilizce_seviyeniz?"
    ] = 0

    # Handle cases where "ingilizce_biliyor_musunuz?" is 1
    df.loc[
        (df["ingilizce_biliyor_musunuz?"] == 1) & (df["ingilizce_seviyeniz?"] == "nan"),
        "ingilizce_seviyeniz?",
    ] = 0

    # Map English level strings to numerical values
    level_map = {"Başlangıç": 1, "Orta": 2, "İleri": 3}
    df["ingilizce_seviyeniz?"] = (
        df["ingilizce_seviyeniz?"].map(level_map).fillna(df["ingilizce_seviyeniz?"])
    )

    return df

In [102]:
train_data = english_level_cleaner(train_data)

In [103]:
train_data["ingilizce_seviyeniz?"].unique()

array([nan,  0.])

In [104]:
train_data["ingilizce_seviyeniz?"] = train_data["ingilizce_seviyeniz?"].fillna(0)

In [105]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,347.0,1.0,...,2.0,Hayır,Evet,Evet,NaN,1,0.0,0,2002,6
1,2023,1,42,42,14.0,0,0,0,432.0,1.0,...,3.0,Hayır,Evet,Hayır,NaN,1,0.0,1,2004,5
2,2023,0,34,34,179.0,1,100,1,44.0,1.0,...,0.0,Hayır,Hayır,Hayır,NaN,1,0.0,2,2002,4
3,2023,0,47,47,171.0,0,0,0,228.0,1.0,...,4.0,Hayır,Hayır,Evet,NaN,0,0.0,3,2003,3
4,2023,0,55,34,78.0,0,0,0,230.0,3.0,...,1.0,Hayır,Hayır,Evet,NaN,1,0.0,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,111.0,2.0,...,0.0,Evet,Hayır,Evet,NaN,1,0.0,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,159.0,3.0,...,3.0,Evet,Hayır,Evet,NaN,0,0.0,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,104.0,1.0,...,2.0,Evet,Evet,Evet,NaN,1,0.0,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,9.0,4.0,...,2.0,Evet,Evet,Evet,NaN,1,0.0,11047,2001,10


In [106]:
train_data.columns

Index(['basvuru_yili', 'cinsiyet', 'dogum_yeri', 'ikametgah_sehri',
       'universite_adi', 'universite_turu', 'burslu_ise_burs_yuzdesi',
       'burs_aliyor_mu?', 'bölüm', 'universite_kacinci_sinif',
       'universite_not_ortalamasi', 'lise_adi', 'lise_sehir',
       'lise_mezuniyet_notu', 'anne_egitim_durumu', 'anne_calisma_durumu',
       'anne_sektor', 'baba_egitim_durumu', 'baba_calisma_durumu',
       'baba_sektor', 'kardes_sayisi',
       'girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?',
       'profesyonel_bir_spor_daliyla_mesgul_musunuz?',
       'aktif_olarak_bir_stk_üyesi_misiniz?', 'stk_projesine_katildiniz_mi?',
       'ingilizce_biliyor_musunuz?', 'ingilizce_seviyeniz?', 'id',
       'dogum_yili', 'dogum_ayi'],
      dtype='object')

In [107]:
train_data["ingilizce_seviyeniz?"] = train_data["ingilizce_seviyeniz?"].astype("float")

In [108]:
train_data["anne_egitim_durumu"].value_counts().to_dict()

{'İlkokul': 3977,
 'Lise': 2362,
 'Üniversite': 1798,
 'Eğitimi yok': 1457,
 'Ortaokul': 1272,
 'Yüksek Lisans': 143,
 'Doktora': 40}

In [109]:
education = {
    "İlkokul": 1,
    "Lise": 3,
    "İLKOKUL MEZUNU": 1,
    "İlkokul Mezunu": 1,
    "Üniversite": 4,
    "LİSE": 3,
    "Eğitimi yok": 1,
    "Ortaokul": 2,
    "ÜNİVERSİTE": 4,
    "EĞİTİM YOK": 1,
    "Ortaokul Mezunu": 2,
    "Eğitim Yok": 1,
    "ORTAOKUL MEZUNU": 2,
    "Lise Mezunu": 3,
    "Üniversite Mezunu": 4,
    "nan": 0,
    "Yüksek Lisans": 5,
    "YÜKSEK LİSANS": 5,
    "Yüksek Lisans / Doktora": 6,
    "Yüksek Lisans / Doktara": 6,
    "Doktora": 6,
    "DOKTORA": 6,
}

In [110]:
train_data["anne_egitim_durumu"] = train_data["anne_egitim_durumu"].map(education)

In [111]:
train_data["anne_egitim_durumu"]

0        1
1        1
2        5
3        1
4        4
        ..
11044    1
11045    1
11046    1
11047    1
11048    1
Name: anne_egitim_durumu, Length: 11049, dtype: int64

In [112]:
train_data["anne_calisma_durumu"].value_counts(dropna=False)

anne_calisma_durumu
Hayır    8691
Evet     2358
Name: count, dtype: int64

In [113]:
train_data["baba_egitim_durumu"]

0          Ortaokul
1           İlkokul
2        Üniversite
3              Lise
4        Üniversite
            ...    
11044      Ortaokul
11045       İlkokul
11046    Üniversite
11047      Ortaokul
11048    Üniversite
Name: baba_egitim_durumu, Length: 11049, dtype: object

In [114]:
train_data["baba_egitim_durumu"] = train_data["baba_egitim_durumu"].map(education)
train_data["baba_egitim_durumu"] = train_data["baba_egitim_durumu"].fillna(0)

In [115]:
train_data["baba_egitim_durumu"].value_counts(dropna=False)

baba_egitim_durumu
1    3571
4    2728
3    2706
2    1666
5     272
6     106
Name: count, dtype: int64

In [116]:
highschools = pd.read_csv(
    "./datathon-dataset/processed-data/highschools_of_istanbul.csv"
)
best_highschools = highschools.iloc[0:500, 0].str.split("/", expand=True)[2]
best_highschools.dropna(inplace=True)
best_highschools = best_highschools.str.strip()
best_highschools = best_highschools.to_dict()
best_highschools

{1: 'Galatasaray Üniversitesi Galatasaray Lisesi',
 2: 'İstanbul Erkek Lisesi',
 3: 'Kabataş Erkek Lisesi',
 4: 'Kabataş Erkek Lisesi',
 5: 'İstanbul Atatürk Fen Lisesi',
 6: 'Cağaloğlu Anadolu Lisesi',
 7: 'Çapa Fen Lisesi',
 8: 'Hüseyin Avni Sözen Anadolu Lisesi',
 9: 'Hüseyin Avni Sözen Anadolu Lisesi',
 10: 'Baykar Milli Teknoloji Mesleki Ve Teknik Anadolu Lisesi',
 11: 'Beşiktaş Sakıp Sabancı Anadolu Lisesi',
 12: 'Kartal Anadolu İmam Hatip Lisesi',
 13: 'Kadıköy Anadolu Lisesi',
 14: 'Haydarpaşa Lisesi',
 15: 'Baykar Milli Teknoloji Mesleki Ve Teknik Anadolu Lisesi',
 16: 'Vefa Lisesi',
 17: 'Haydarpaşa Lisesi',
 18: 'Yaşar Acar Fen Lisesi',
 19: 'Teknopark İstanbul Mesleki Ve Teknik Anadolu Lisesi',
 20: 'Burak Bora Anadolu Lisesi',
 21: 'Beşiktaş Anadolu Lisesi',
 22: 'Burak Bora Anadolu Lisesi',
 23: 'Validebağ Fen Lisesi',
 24: 'Kadıköy Anadolu İmam Hatip Lisesi',
 25: 'Pertevniyal Lisesi',
 26: 'Şehit Münir Alkan Fen Lisesi',
 27: 'İstanbul Ticaret Odası Marmara Anadolu İmam

In [117]:
threshold = 90


def is_similar_to_highschool(row):
    for key in best_highschools:
        if fuzz.partial_ratio(str(row), best_highschools[key]) > threshold:
            return True
    return False

In [118]:
train_data["lise_adi"] = train_data["lise_adi"].apply(
    lambda x: is_similar_to_highschool(x)
)

In [119]:
train_data["lise_adi"].value_counts()

lise_adi
False    9663
True     1386
Name: count, dtype: int64

In [120]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,347.0,1.0,...,2.0,Hayır,Evet,Evet,NaN,1,0.0,0,2002,6
1,2023,1,42,42,14.0,0,0,0,432.0,1.0,...,3.0,Hayır,Evet,Hayır,NaN,1,0.0,1,2004,5
2,2023,0,34,34,179.0,1,100,1,44.0,1.0,...,0.0,Hayır,Hayır,Hayır,NaN,1,0.0,2,2002,4
3,2023,0,47,47,171.0,0,0,0,228.0,1.0,...,4.0,Hayır,Hayır,Evet,NaN,0,0.0,3,2003,3
4,2023,0,55,34,78.0,0,0,0,230.0,3.0,...,1.0,Hayır,Hayır,Evet,NaN,1,0.0,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,111.0,2.0,...,0.0,Evet,Hayır,Evet,NaN,1,0.0,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,159.0,3.0,...,3.0,Evet,Hayır,Evet,NaN,0,0.0,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,104.0,1.0,...,2.0,Evet,Evet,Evet,NaN,1,0.0,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,9.0,4.0,...,2.0,Evet,Evet,Evet,NaN,1,0.0,11047,2001,10


In [121]:
train_data["lise_adi"].value_counts(dropna=False)

lise_adi
False    9663
True     1386
Name: count, dtype: int64

In [122]:
train_data["lise_mezuniyet_notu"].value_counts()

lise_mezuniyet_notu
75 - 100    9742
50 - 74     1301
25 - 49        6
Name: count, dtype: int64

In [123]:
highschool_score = {
    "75 - 100": 4,
    "84-70": 4,
    "100-85": 4,
    "nan": pd.NA,
    "4.00-3.50": 4,
    "3.50-3.00": 4,
    "50 - 75": 3,
    "3.00 - 4.00": 4,
    "69-55": 3,
    "3.00-2.50": 3,
    "3.50-3": 4,
    "50 - 74": 3,
    "2.50 ve altı": 3,
    "54-45": 2,
    "Not ortalaması yok": pd.NA,
    "25 - 50": 2,
    "44-0": pd.NA,
    "25 - 49": 2,
    "0 - 25": 1,
    "0 - 24": 1,
}

In [124]:
train_data["lise_mezuniyet_notu"] = train_data["lise_mezuniyet_notu"].map(
    highschool_score
)
train_data["lise_mezuniyet_notu"] = pd.to_numeric(
    train_data["lise_mezuniyet_notu"], errors="coerce"
)
train_data["lise_mezuniyet_notu"].fillna(0, inplace=True)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16198/4036599635.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["lise_mezuniyet_notu"].fillna(0, inplace=True)


In [125]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,347.0,1.0,...,2.0,Hayır,Evet,Evet,NaN,1,0.0,0,2002,6
1,2023,1,42,42,14.0,0,0,0,432.0,1.0,...,3.0,Hayır,Evet,Hayır,NaN,1,0.0,1,2004,5
2,2023,0,34,34,179.0,1,100,1,44.0,1.0,...,0.0,Hayır,Hayır,Hayır,NaN,1,0.0,2,2002,4
3,2023,0,47,47,171.0,0,0,0,228.0,1.0,...,4.0,Hayır,Hayır,Evet,NaN,0,0.0,3,2003,3
4,2023,0,55,34,78.0,0,0,0,230.0,3.0,...,1.0,Hayır,Hayır,Evet,NaN,1,0.0,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,111.0,2.0,...,0.0,Evet,Hayır,Evet,NaN,1,0.0,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,159.0,3.0,...,3.0,Evet,Hayır,Evet,NaN,0,0.0,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,104.0,1.0,...,2.0,Evet,Evet,Evet,NaN,1,0.0,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,9.0,4.0,...,2.0,Evet,Evet,Evet,NaN,1,0.0,11047,2001,10


In [126]:
train_data["lise_mezuniyet_notu"].value_counts(dropna=False)

lise_mezuniyet_notu
4    9742
3    1301
2       6
Name: count, dtype: int64

In [127]:
working_situation_map = {"Hayır": 0, "nan": 0, "Evet": 1, "Emekli": 2}

In [128]:
train_data["anne_calisma_durumu"] = train_data["anne_calisma_durumu"].map(
    working_situation_map
)
train_data["anne_calisma_durumu"] = train_data["anne_calisma_durumu"].fillna(0)

In [129]:
train_data["baba_calisma_durumu"] = train_data["baba_calisma_durumu"].map(
    working_situation_map
)
train_data["baba_calisma_durumu"] = train_data["baba_calisma_durumu"].fillna(0)

In [130]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,347.0,1.0,...,2.0,Hayır,Evet,Evet,NaN,1,0.0,0,2002,6
1,2023,1,42,42,14.0,0,0,0,432.0,1.0,...,3.0,Hayır,Evet,Hayır,NaN,1,0.0,1,2004,5
2,2023,0,34,34,179.0,1,100,1,44.0,1.0,...,0.0,Hayır,Hayır,Hayır,NaN,1,0.0,2,2002,4
3,2023,0,47,47,171.0,0,0,0,228.0,1.0,...,4.0,Hayır,Hayır,Evet,NaN,0,0.0,3,2003,3
4,2023,0,55,34,78.0,0,0,0,230.0,3.0,...,1.0,Hayır,Hayır,Evet,NaN,1,0.0,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,111.0,2.0,...,0.0,Evet,Hayır,Evet,NaN,1,0.0,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,159.0,3.0,...,3.0,Evet,Hayır,Evet,NaN,0,0.0,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,104.0,1.0,...,2.0,Evet,Evet,Evet,NaN,1,0.0,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,9.0,4.0,...,2.0,Evet,Evet,Evet,NaN,1,0.0,11047,2001,10


In [131]:
train_data["baba_calisma_durumu"].value_counts(dropna=False)

baba_calisma_durumu
1    6177
0    4872
Name: count, dtype: int64

In [132]:
train_data["anne_calisma_durumu"].value_counts(dropna=False)

anne_calisma_durumu
0    8691
1    2358
Name: count, dtype: int64

In [133]:
sector_map = {
    "0": 0,
    "-": 0,
    "nan": 0,
    "Kamu": 1,
    "KAMU": 1,
    "Özel Sektör": 2,
    "ÖZEL SEKTÖR": 2,
    "Diğer": 3,
    "DİĞER": 3,
}

In [134]:
train_data["anne_sektor"] = train_data["anne_sektor"].map(sector_map)
train_data["anne_sektor"] = train_data["anne_sektor"].fillna(0)

In [135]:
train_data["anne_sektor"].value_counts(dropna=False)

anne_sektor
0    8629
2    1112
1     989
3     319
Name: count, dtype: int64

In [136]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,347.0,1.0,...,2.0,Hayır,Evet,Evet,NaN,1,0.0,0,2002,6
1,2023,1,42,42,14.0,0,0,0,432.0,1.0,...,3.0,Hayır,Evet,Hayır,NaN,1,0.0,1,2004,5
2,2023,0,34,34,179.0,1,100,1,44.0,1.0,...,0.0,Hayır,Hayır,Hayır,NaN,1,0.0,2,2002,4
3,2023,0,47,47,171.0,0,0,0,228.0,1.0,...,4.0,Hayır,Hayır,Evet,NaN,0,0.0,3,2003,3
4,2023,0,55,34,78.0,0,0,0,230.0,3.0,...,1.0,Hayır,Hayır,Evet,NaN,1,0.0,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,111.0,2.0,...,0.0,Evet,Hayır,Evet,NaN,1,0.0,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,159.0,3.0,...,3.0,Evet,Hayır,Evet,NaN,0,0.0,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,104.0,1.0,...,2.0,Evet,Evet,Evet,NaN,1,0.0,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,9.0,4.0,...,2.0,Evet,Evet,Evet,NaN,1,0.0,11047,2001,10


In [137]:
train_data["baba_sektor"].value_counts()

baba_sektor
-              4752
Özel Sektör    2793
Diğer          1804
Kamu           1700
Name: count, dtype: int64

In [138]:
train_data["baba_sektor"] = train_data["baba_sektor"].map(sector_map)
train_data["baba_sektor"] = train_data["baba_sektor"].fillna(0)

In [139]:
train_data["baba_sektor"].value_counts(dropna=False)

baba_sektor
0    4752
2    2793
3    1804
1    1700
Name: count, dtype: int64

In [140]:
train_data["kardes_sayisi"].value_counts(dropna=False)

kardes_sayisi
1.0    3612
2.0    2821
4.0    2421
3.0    1490
0.0     705
Name: count, dtype: int64

In [141]:
a = train_data["kardes_sayisi"].copy()

In [142]:
train_data["kardes_sayisi"] = pd.to_numeric(
    train_data["kardes_sayisi"], errors="coerce"
)
train_data["kardes_sayisi"] = train_data["kardes_sayisi"].fillna(0)

In [143]:
train_data["kardes_sayisi"].value_counts(dropna=False)

kardes_sayisi
1.0    3612
2.0    2821
4.0    2421
3.0    1490
0.0     705
Name: count, dtype: int64

In [144]:
train_data["lise_adi"] = train_data["lise_adi"].apply(lambda x: 1 if x == True else 0)

In [145]:
train_data.rename(
    columns={"lise_adi": "en_iyi_500_lise_içerisinde_mi_eğitim_gördü"}, inplace=True
)

In [146]:
first_part = train_data.copy()

In [147]:
train_data = first_part.copy()

In [148]:
train_data

,basvuru_yili,cinsiyet,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu?,bölüm,universite_kacinci_sinif,...,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?,profesyonel_bir_spor_daliyla_mesgul_musunuz?,aktif_olarak_bir_stk_üyesi_misiniz?,stk_projesine_katildiniz_mi?,ingilizce_biliyor_musunuz?,ingilizce_seviyeniz?,id,dogum_yili,dogum_ayi
0,2023,1,5,5,199.0,0,0,0,347.0,1.0,...,2.0,Hayır,Evet,Evet,NaN,1,0.0,0,2002,6
1,2023,1,42,42,14.0,0,0,0,432.0,1.0,...,3.0,Hayır,Evet,Hayır,NaN,1,0.0,1,2004,5
2,2023,0,34,34,179.0,1,100,1,44.0,1.0,...,0.0,Hayır,Hayır,Hayır,NaN,1,0.0,2,2002,4
3,2023,0,47,47,171.0,0,0,0,228.0,1.0,...,4.0,Hayır,Hayır,Evet,NaN,0,0.0,3,2003,3
4,2023,0,55,34,78.0,0,0,0,230.0,3.0,...,1.0,Hayır,Hayır,Evet,NaN,1,0.0,4,2002,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,1,66,6,13.0,0,0,0,111.0,2.0,...,0.0,Evet,Hayır,Evet,NaN,1,0.0,11044,2002,1
11045,2023,1,42,42,180.0,0,0,0,159.0,3.0,...,3.0,Evet,Hayır,Evet,NaN,0,0.0,11045,2001,9
11046,2023,0,6,6,146.0,0,0,0,104.0,1.0,...,2.0,Evet,Evet,Evet,NaN,1,0.0,11046,2004,6
11047,2023,1,2,34,50.0,0,0,0,9.0,4.0,...,2.0,Evet,Evet,Evet,NaN,1,0.0,11047,2001,10


In [149]:
train_data["lise_sehir"] = train_data["lise_sehir"].apply(clean_city_name)

In [150]:
train_data["lise_sehir"].value_counts(dropna=False)

lise_sehir
34    1998
6      779
31     428
35     390
21     375
      ... 
8       16
29      15
74      15
75      11
69       7
Name: count, Length: 80, dtype: int64

In [151]:
train_data["lise_sehir"].fillna(train_data["lise_sehir"].mode()[0], inplace=True)

/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16198/3314362712.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["lise_sehir"].fillna(train_data["lise_sehir"].mode()[0], inplace=True)
/var/folders/4k/j4r1d2y94nq363xgn__kwzzw0000gn/T/ipykernel_16198/3314362712.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_sile

In [152]:
club_map = {"Hayır": 0, "nan": 0, "Evet": 1}

In [153]:
train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"].value_counts(
    dropna=False
)

girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?
Hayır    5940
Evet     5109
Name: count, dtype: int64

In [154]:
train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"]

0        Hayır
1        Hayır
2        Hayır
3        Hayır
4        Hayır
         ...  
11044     Evet
11045     Evet
11046     Evet
11047     Evet
11048    Hayır
Name: girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?, Length: 11049, dtype: object

In [155]:
train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"] = train_data[
    "girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"
].map(club_map)

train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz"] = pd.to_numeric(
    train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"], errors="coerce"
)
train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"] = train_data[
    "girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz?"
].fillna(0)

In [156]:
train_data["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz"].value_counts(
    dropna=False
)

girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz
0    5940
1    5109
Name: count, dtype: int64

In [157]:
do_you_do_sports = club_map.copy()

In [158]:
train_data["profesyonel_bir_spor_daliyla_mesgul_musunuz?"] = train_data[
    "profesyonel_bir_spor_daliyla_mesgul_musunuz?"
].map(do_you_do_sports)

train_data["profesyonel_bir_spor_daliyla_mesgul_musunuz?"] = pd.to_numeric(
    train_data["profesyonel_bir_spor_daliyla_mesgul_musunuz?"], errors="coerce"
)
train_data["profesyonel_bir_spor_daliyla_mesgul_musunuz?"] = train_data[
    "profesyonel_bir_spor_daliyla_mesgul_musunuz?"
].fillna(0)

In [159]:
train_data["profesyonel_bir_spor_daliyla_mesgul_musunuz?"].value_counts(dropna=False)

profesyonel_bir_spor_daliyla_mesgul_musunuz?
0    6422
1    4627
Name: count, dtype: int64

In [160]:
are_you_in_a_stk_map = do_you_do_sports.copy()

In [161]:
train_data["aktif_olarak_bir_stk_üyesi_misiniz?"] = train_data[
    "aktif_olarak_bir_stk_üyesi_misiniz?"
].map(do_you_do_sports)

train_data["aktif_olarak_bir_stk_üyesi_misiniz?"] = pd.to_numeric(
    train_data["aktif_olarak_bir_stk_üyesi_misiniz?"], errors="coerce"
)
train_data["aktif_olarak_bir_stk_üyesi_misiniz?"] = train_data[
    "aktif_olarak_bir_stk_üyesi_misiniz?"
].fillna(0)

In [162]:
are_you_a_stk_member_map = are_you_in_a_stk_map.copy()

In [163]:
train_data["stk_projesine_katildiniz_mi?"] = train_data[
    "stk_projesine_katildiniz_mi?"
].map(do_you_do_sports)

train_data["stk_projesine_katildiniz_mi?"] = pd.to_numeric(
    train_data["stk_projesine_katildiniz_mi?"], errors="coerce"
)
train_data["stk_projesine_katildiniz_mi?"] = train_data[
    "stk_projesine_katildiniz_mi?"
].fillna(0)

In [164]:
train_data.to_csv("./datathon-dataset/processed-data/test_data_cleaned.csv")